# Airline Customer Loyalty Program Analysis
A notebook by Mobolaji Salawu

## 1. Import Libraries

In [1]:
import numpy as np
import pandas as pd
import bokeh
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource

output_notebook()

## 2. Import the Customer Flight Activity and Customer Loyalty History Datasets

In [2]:
# Import Customer Flight Activity dataset
# Read Loyalty Number, Year and Month columns as string

loyalty_flight = pd.read_csv("Customer Flight Activity.csv", dtype = {"Loyalty Number": str, "Year": str, "Month": str})
loyalty_flight.head()

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0,100018,2017,1,3,0,3,1521,152.0,0,0
1,100102,2017,1,10,4,14,2030,203.0,0,0
2,100140,2017,1,6,0,6,1200,120.0,0,0
3,100214,2017,1,0,0,0,0,0.0,0,0
4,100272,2017,1,0,0,0,0,0.0,0,0


In [3]:
# Import Customer Loyalty History dataset

loyalty_history = pd.read_csv("Customer Loyalty History.csv", dtype = {"Loyalty Number": str, "Enrollment Year": str,
                              "Enrollment Month": str, "Cancellation Year": str, "Cancellation Month": str})
loyalty_history.head()

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
0,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN
1,549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,NaN,Divorced,Star,3839.61,Standard,2016,3,NaN,NaN
2,429460,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Single,Star,3839.75,Standard,2014,7,2018,1
3,608370,Canada,Ontario,Toronto,P1W 1K4,Male,College,NaN,Single,Star,3839.75,Standard,2013,2,NaN,NaN
4,530508,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,103495.0,Married,Star,3842.79,Standard,2014,10,NaN,NaN


In [4]:
# Take a look at impoorted datasets

print("Customer flight activity \n")
print(loyalty_flight.info(), "\n")
print("Customer loyalty history \n")
print(loyalty_history.info())

Customer flight activity 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405624 entries, 0 to 405623
Data columns (total 10 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Loyalty Number               405624 non-null  object 
 1   Year                         405624 non-null  object 
 2   Month                        405624 non-null  object 
 3   Flights Booked               405624 non-null  int64  
 4   Flights with Companions      405624 non-null  int64  
 5   Total Flights                405624 non-null  int64  
 6   Distance                     405624 non-null  int64  
 7   Points Accumulated           405624 non-null  float64
 8   Points Redeemed              405624 non-null  int64  
 9   Dollar Cost Points Redeemed  405624 non-null  int64  
dtypes: float64(1), int64(6), object(3)
memory usage: 30.9+ MB
None 

Customer loyalty history 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16737 e

## 3. Data Cleaning and Transformation

In [5]:
# Examine the numeric columns of loyalty_fight

print("Customer flight activity \n")
loyalty_flight.describe()

Customer flight activity 



,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
count,405624.000000,405624.000000,405624.000000,405624.000000,405624.000000,405624.000000,405624.000000
mean,4.115052,1.031805,5.146858,1208.880059,123.692721,30.696872,2.484503
std,5.225518,2.076869,6.521227,1433.155320,146.599831,125.486049,10.150038
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,1.000000,488.000000,50.000000,0.000000,0.000000
75%,8.000000,1.000000,10.000000,2336.000000,239.000000,0.000000,0.000000
max,21.000000,11.000000,32.000000,6293.000000,676.500000,876.000000,71.000000


In [6]:
# Examine the numeric columns of loyalty_history
# Seems some loyalty members have negative salary. We can ignore this because we will not use this column in our analysis

print("Customer loyalty history \n")
loyalty_history.describe()

Customer loyalty history 



,Salary,CLV
count,12499.000000,16737.000000
mean,79245.609409,7988.896536
std,35008.297285,6860.982280
min,-58486.000000,1898.010000
25%,59246.500000,3980.840000
50%,73455.000000,5780.180000
75%,88517.500000,8940.580000
max,407228.000000,83325.380000


In [7]:
# For fun, we can check the number of records affected by negative salary

len(loyalty_history[loyalty_history['Salary'] < 0].index)

20

In [8]:
# Earlier info() showed that some columns in loyalty_history contain NaN

loyalty_history.isna().sum()

,0
Loyalty Number,0
Country,0
Province,0
City,0
Postal Code,0
Gender,0
Education,0
Salary,4238
Marital Status,0
Loyalty Card,0


In [9]:
# Check loyalty_flight for NaN values too

loyalty_flight.isnull().sum()

,0
Loyalty Number,0
Year,0
Month,0
Flights Booked,0
Flights with Companions,0
Total Flights,0
Distance,0
Points Accumulated,0
Points Redeemed,0
Dollar Cost Points Redeemed,0


In [10]:
# Replace NaN with blank in loyalty_history
# This also casts Salary column as string
# Can also use df.fillna()

loyalty_history.replace(np.nan, "", inplace=True)
loyalty_history.isna().sum()

,0
Loyalty Number,0
Country,0
Province,0
City,0
Postal Code,0
Gender,0
Education,0
Salary,0
Marital Status,0
Loyalty Card,0


In [11]:
# Examine the non-numeric columns of loyalty_fight with the exception of the secondary key column, Loyalty Number

nonnumeric_columns = ["Year", "Month"]
for column in nonnumeric_columns:
    print(loyalty_flight[column].value_counts(), "\n")

Year
2017    202812
2018    202812
Name: count, dtype: int64 

Month
1     33802
9     33802
2     33802
3     33802
11    33802
4     33802
5     33802
7     33802
6     33802
8     33802
10    33802
12    33802
Name: count, dtype: int64 



In [12]:
# Break non-numeric columns in loyalty_history into three as taking all non-numeric columns at a go will give a long output

address_columns = ["Country", "Province", "City"]
demographic_columns = ["Gender", "Education", "Marital Status", "Loyalty Card", "Enrollment Type"]
yearmonth_columns = ["Enrollment Year", "Enrollment Month", "Cancellation Year", "Cancellation Month"]

In [13]:
# Examine the address columns of loyalty_history

for add in address_columns:
    print(loyalty_history[add].value_counts(), "\n")

Country
Canada    16737
Name: count, dtype: int64 

Province
Ontario                 5404
British Columbia        4409
Quebec                  3300
Alberta                  969
Manitoba                 658
New Brunswick            636
Nova Scotia              518
Saskatchewan             409
Newfoundland             258
Yukon                    110
Prince Edward Island      66
Name: count, dtype: int64 

City
Toronto           3351
Vancouver         2582
Montreal          2059
Winnipeg           658
Whistler           582
Halifax            518
Ottawa             509
Trenton            486
Edmonton           486
Quebec City        485
Dawson Creek       444
Fredericton        425
Regina             409
Kingston           401
Tremblant          398
Victoria           389
Hull               358
West Vancouver     324
St. John's         258
Thunder Bay        256
Sudbury            227
Moncton            211
Calgary            191
Banff              179
London             174
Peace River 

In [14]:
# Examine the demography columns of loyalty_history

for demo in demographic_columns:
    print(loyalty_history[demo].value_counts(), "\n")

Gender
Female    8410
Male      8327
Name: count, dtype: int64 

Education
Bachelor                10475
College                  4238
High School or Below      782
Doctor                    734
Master                    508
Name: count, dtype: int64 

Marital Status
Married     9735
Single      4484
Divorced    2518
Name: count, dtype: int64 

Loyalty Card
Star      7637
Nova      5671
Aurora    3429
Name: count, dtype: int64 

Enrollment Type
Standard          15766
2018 Promotion      971
Name: count, dtype: int64 



In [15]:
# Examine the year and month columns of loyalty_history

for yrmnth in yearmonth_columns:
    print(loyalty_history[yrmnth].value_counts(), "\n")

Enrollment Year
2018    3010
2017    2487
2016    2456
2013    2397
2014    2370
2015    2331
2012    1686
Name: count, dtype: int64 

Enrollment Month
5     1503
12    1480
7     1473
11    1446
10    1444
8     1430
6     1412
9     1391
4     1388
3     1358
2     1220
1     1192
Name: count, dtype: int64 

Cancellation Year
        14670
2018      645
2017      506
2016      427
2015      265
2014      181
2013       43
Name: count, dtype: int64 

Cancellation Month
      14670
12      213
11      212
8       208
7       186
10      180
9       176
6       165
1       155
3       149
5       148
2       139
4       136
Name: count, dtype: int64 



In [16]:
# Derive the total flights and points for each customer from loyalty_flight

cust_flight_point = loyalty_flight.drop(["Year", "Month", "Dollar Cost Points Redeemed"], axis = "columns")
cust_flight_point = cust_flight_point.groupby("Loyalty Number").sum().reset_index()
cust_flight_point.head()

,Loyalty Number,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed
0,100018,157,35,192,50682,5376.00,1513
1,100102,173,42,215,40222,4115.25,1195
2,100140,152,38,190,41252,4184.25,593
3,100214,79,17,96,33982,3426.00,861
4,100272,127,36,163,40872,4108.04,1007


In [17]:
# Determine customers that belong to the high value segment
# These are customers with total flights at or above the 75th percentile of the total population

cust_high_value = cust_flight_point[cust_flight_point["Total Flights"] >= cust_flight_point["Total Flights"].quantile(0.75)]
cust_high_value.head()

,Loyalty Number,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed
0,100018,157,35,192,50682,5376.00,1513
1,100102,173,42,215,40222,4115.25,1195
2,100140,152,38,190,41252,4184.25,593
5,100301,145,46,191,42594,4273.00,1381
10,100550,146,37,183,41157,4235.50,1114


In [18]:
# Filter loyalty_history to include only customers in the high value segment
# Make this a separate dataframe

loyalty_history_high_value = loyalty_history.merge(cust_high_value[["Loyalty Number", "Total Flights"]], on="Loyalty Number", how="right")
loyalty_history_high_value.head()

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month,Total Flights
0,100018,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8,,,192
1,100102,Canada,Ontario,Toronto,M1R 4K3,Male,College,,Single,Nova,2887.74,Standard,2013,3,,,215
2,100140,Canada,British Columbia,Dawson Creek,U5I 4F1,Female,College,,Divorced,Nova,2838.07,Standard,2016,7,,,190
3,100301,Canada,Ontario,Toronto,P1J 8T7,Male,Bachelor,70323.0,Divorced,Nova,48356.96,Standard,2013,9,,,191
4,100550,Canada,Quebec,Montreal,H2Y 4R4,Female,Bachelor,54133.0,Married,Nova,7861.80,Standard,2013,2,,,183


In [19]:
# Filter loyalty_flight to include only customers in the high value segment
# Make this a separate dataframe

loyalty_flight_high_value = loyalty_flight.merge(cust_high_value[["Loyalty Number", "Total Flights"]], on="Loyalty Number", how="right")
loyalty_flight_high_value.head()

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights_x,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed,Total Flights_y
0,100018,2017,1,3,0,3,1521,152.0,0,0,192
1,100018,2017,2,2,2,4,1320,132.0,0,0,192
2,100018,2018,10,6,4,10,3110,311.0,385,31,192
3,100018,2017,4,4,0,4,924,92.0,0,0,192
4,100018,2017,5,0,0,0,0,0.0,0,0,192


## 4. Exploratory Data Analysis

### Exploration of Trends and Insights from the Loyalty Program

In [20]:
# Trend of enrollment into the loyalty program
# Loyalty members that enrolled by year regardless of whether they stayed or cancelled

enrollment_by_year = loyalty_history.groupby('Enrollment Year').size()

# Prepare data
year = enrollment_by_year.index.values
member_count = enrollment_by_year.values
source = ColumnDataSource(data={"year":year, "enrolled": member_count})

# Create line chart
plot = figure(title="Loyalty Members Enrollment Trend \n", x_range=year, y_range=(0, max(member_count)+100),
              x_axis_label='Year', y_axis_label="Number of Loyalty Members", tooltips="Total Enrolled: @enrolled",
              width=800, height=400)
plot.line(x="year", y="enrolled", line_width=2, legend_label="Enrolled", color="navy", source=source)

# Customize the line chart
plot.legend.location = "top_left"
plot.legend.orientation = "horizontal"
plot.xgrid.grid_line_color = None

show(plot)

In [21]:
# New customers vs lost customers vs net gain/loss

from bokeh.models import HoverTool

lost_customers = loyalty_history[loyalty_history["Cancellation Year"] != ""]
cancellation_by_year = lost_customers.groupby("Cancellation Year").size()

# Merge enrollment_by_year and cancellation_by_year into a dataframe object, replace NaN with 0
loyalty_net = pd.concat([enrollment_by_year, cancellation_by_year], axis=1).reset_index().fillna(0)

# Give columns recognizable names, add net_gained_loss column
loyalty_net.rename(columns={"index":"Year", 0:"Total Enrolled", 1:"Total Cancelled"}, inplace=True)
loyalty_net["net_gained_loss"] = loyalty_net["Total Enrolled"] - loyalty_net["Total Cancelled"]

# Prepare data
year = loyalty_net["Year"]
enrolled = loyalty_net["Total Enrolled"]
cancelled = loyalty_net["Total Cancelled"]*-1
gain_loss = loyalty_net["net_gained_loss"]
data = {"year":year, "enrolled":enrolled, "cancelled":cancelled, "gain_loss":gain_loss}

# Create bar and line charts
plot_mix = figure(title="Loyalty Members Enrolled against Loyalty Members Lost \n", x_range=year, y_range=(min(cancelled), max(enrolled)+100),
              x_axis_label='Year', y_axis_label="Number of Loyalty Members",
              width=800, height=500)
bar1 = plot_mix.vbar(x="year", top="enrolled", width=0.7, legend_label="Total Enrolled", fill_color="green", source=data)
bar2 = plot_mix.vbar(x="year", top="cancelled", width=0.7, legend_label="Total Cancelled", fill_color="red", source=data)
line1 = plot_mix.line(x="year", y="gain_loss", line_width=2, legend_label="Net Retained", color="navy", source=data)
circle1 = plot_mix.scatter(x="year", y="gain_loss", size=5, legend_label="Net Retained", color="navy", source=data)

plot_mix.add_tools(HoverTool(renderers=[bar1], tooltips=[("Total Enrolled", "@enrolled")]),
                HoverTool(renderers=[bar2], tooltips=[("Total Cancelled", "@cancelled")]),
                HoverTool(renderers=[circle1], tooltips=[("Net Retained", "@gain_loss")]))

# Customize the chart
plot_mix.legend.location = "top_left"
plot_mix.legend.orientation = "horizontal"
plot_mix.legend.label_text_font_size = "8.5pt"
plot_mix.xgrid.grid_line_color = None

show(plot_mix)

In [22]:
# Active members vs current members

from bokeh.transform import dodge

# Current members in 2017, 2018
current_members_2017 = len(loyalty_history[(loyalty_history["Cancellation Year"]=="") | (loyalty_history["Cancellation Year"]=="2018")])
current_members_2018 = len(loyalty_history[loyalty_history["Cancellation Year"]==""])

# Get the number of loyalty members that booked flight in 2017 and 2018
active_members = loyalty_flight[loyalty_flight["Flights Booked"] != 0].groupby("Year")
active_2017 = active_members.get_group("2017")["Loyalty Number"].nunique()
active_2018 = active_members.get_group("2018")["Loyalty Number"].nunique()

# Prepare data
# Create dataframe object
data = {"Year": ["2017", "2018"], "Current Members": [current_members_2017, current_members_2018], "Active Members": [active_2017, active_2018]}
loyalty_active = pd.DataFrame(data=data)

year = loyalty_active["Year"]
current = loyalty_active["Current Members"]
active = loyalty_active["Active Members"]

data = {"year": year, "current": current, "active": active}

source = ColumnDataSource(data=data)

# Create clustered bar chart
plot_c = figure(title="Active Loyalty Members vs Current Loyalty Members \n", x_range=year, y_range=(0, max(current)+1000),
              x_axis_label='Year', y_axis_label="Number of Loyalty Members",
              width=800, height=500)
bar1 = plot_c.vbar(x=dodge("year", -0.25, range=plot_c.x_range), top="active", source=source, width=0.2, legend_label="Active Members", fill_color="blue")
bar2 = plot_c.vbar(x=dodge("year", 0.0, range=plot_c.x_range), top="current", source=source, width=0.2, legend_label="Current Members", fill_color="navy")

plot_c.add_tools(HoverTool(renderers=[bar1], tooltips=[("Active Members", "@active")]),
                HoverTool(renderers=[bar2], tooltips=[("Current Members", "@current")]))

# Customize the chart
plot_c.legend.location = "top_right"
plot_c.legend.label_text_font_size = "7pt"
plot_c.xgrid.grid_line_color = None

show(plot_c)

In [23]:
# Point earned vs point redeemed vs points balance

# Add Points Balance column to loyalty_flight
loyalty_flight["Points Balance"] = loyalty_flight["Points Accumulated"] - loyalty_flight["Points Redeemed"]

# Get the points accummulated, redeemed and balance in each month of each year; add Year_Month and Sort columns too
points = pd.DataFrame(loyalty_flight.groupby(["Year", "Month"])[["Points Accumulated", "Points Redeemed", "Points Balance"]].sum().reset_index())

points["Year_Month"] = points["Year"] + "-" + points["Month"]
points["Sort"] = points["Year"].astype(int)*100 + points["Month"].astype(int)

# Order by the Sort column
points = points.sort_values("Sort")

# Prepare data
yearmnth = points["Year_Month"]
accumulated = points["Points Accumulated"]
redeemed = points["Points Redeemed"]*-1
balance = points["Points Balance"]
data = {"yearmnth":yearmnth ,"accumulated":accumulated ,"redeemed":redeemed ,"balance":balance}

# Create bar and line charts
plot_p = figure(title="Loyalty Points Earned against Loyal Points Redeemed \n", x_range=yearmnth, y_range=(min(redeemed), max(accumulated)+40000),
              x_axis_label='Year-Month', y_axis_label="Loyalty Points",
              width=1200, height=600)
bar1 = plot_p.vbar(x="yearmnth", top="accumulated", width=0.7, legend_label="Total Points Accumulated", fill_color="green", source=data)
bar2 = plot_p.vbar(x="yearmnth", top="redeemed", width=0.7, legend_label="Total Points Redeemed", fill_color="red", source=data)
line1 = plot_p.line(x="yearmnth", y="balance", line_width=2, legend_label="Points Balance", color="navy", source=data)
circle1 = plot_p.scatter(x="yearmnth", y="balance", size=5, legend_label="Points Balance", color="navy", source=data)

plot_p.add_tools(HoverTool(renderers=[bar1], tooltips=[("Total Points Accumulated", "@accumulated")]),
                HoverTool(renderers=[bar2], tooltips=[("Total Points Redeemed", "@redeemed")]),
                HoverTool(renderers=[circle1], tooltips=[("Points Balance", "@balance")]))

# Customize the chart
plot_p.legend.location = "top_left"
plot_p.legend.orientation = "horizontal"
plot_p.legend.label_text_font_size = "9pt"
plot_p.xgrid.grid_line_color = None
plot_p.xaxis.major_label_orientation = 45

show(plot_p)

### High Value Segment Analysis

In [24]:
# Point earned vs point redeemed vs points balance for high value customer segment

# Points Balance column to loyalty_flight_high_value
loyalty_flight_high_value["Points Balance"] = loyalty_flight_high_value["Points Accumulated"] - loyalty_flight_high_value["Points Redeemed"]

# Get the points accummulated, redeemed and balance in each month of each year; add Year_Month and Sort columns too
points = pd.DataFrame(loyalty_flight_high_value.groupby(["Year", "Month"])[["Points Accumulated", "Points Redeemed", "Points Balance"]].sum().reset_index())

points["Year_Month"] = points["Year"] + "-" + points["Month"]
points["Sort"] = points["Year"].astype(int)*100 + points["Month"].astype(int)

# Order by the Sort column
points = points.sort_values("Sort")

# Prepare data
yearmnth = points["Year_Month"]
accumulated = points["Points Accumulated"]
redeemed = points["Points Redeemed"]*-1
balance = points["Points Balance"]
data = {"yearmnth":yearmnth ,"accumulated":accumulated ,"redeemed":redeemed ,"balance":balance}

# Create bar and line charts
plot_p = figure(title="High Value Customer Segment: Loyalty Points Earned against Loyal Points Redeemed \n", x_range=yearmnth, y_range=(min(redeemed), max(accumulated)+40000),
              x_axis_label='Year-Month', y_axis_label="Loyalty Points",
              width=1200, height=600)
bar1 = plot_p.vbar(x="yearmnth", top="accumulated", width=0.7, legend_label="Total Points Accumulated", fill_color="green", source=data)
bar2 = plot_p.vbar(x="yearmnth", top="redeemed", width=0.7, legend_label="Total Points Redeemed", fill_color="red", source=data)
line1 = plot_p.line(x="yearmnth", y="balance", line_width=2, legend_label="Points Balance", color="navy", source=data)
circle1 = plot_p.scatter(x="yearmnth", y="balance", size=5, legend_label="Points Balance", color="navy", source=data)

plot_p.add_tools(HoverTool(renderers=[bar1], tooltips=[("Total Points Accumulated", "@accumulated")]),
                HoverTool(renderers=[bar2], tooltips=[("Total Points Redeemed", "@redeemed")]),
                HoverTool(renderers=[circle1], tooltips=[("Points Balance", "@balance")]))

# Customize the chart
plot_p.legend.location = "top_left"
plot_p.legend.orientation = "horizontal"
plot_p.legend.label_text_font_size = "9pt"
plot_p.xgrid.grid_line_color = None
plot_p.xaxis.major_label_orientation = 45

show(plot_p)

In [25]:
# Breakdown of high value segment by loyalty card, education, marital status

from math import pi
from bokeh.transform import cumsum
from bokeh.layouts import row

# Group by Loyalty Card
hvs_loyalty_by_card = pd.DataFrame(loyalty_history_high_value.groupby(["Loyalty Card"]).size().reset_index())
hvs_loyalty_by_card.rename(columns={0:"Total Enrolled"}, inplace=True)

# Group by Education
hvs_loyalty_by_ed = pd.DataFrame(loyalty_history_high_value.groupby(["Education"]).size().reset_index())
hvs_loyalty_by_ed.rename(columns={0:"Total Enrolled"}, inplace=True)

# Group by Marital Status
hvs_loyalty_by_ms = pd.DataFrame(loyalty_history.groupby(["Marital Status"]).size().reset_index())
hvs_loyalty_by_ms.rename(columns={0:"Total Enrolled"}, inplace=True)


# Create pie chart for loyalty card
color = ["lightskyblue", "blue", "navy"]
total = hvs_loyalty_by_card["Total Enrolled"].sum()
angle = hvs_loyalty_by_card["Total Enrolled"]/hvs_loyalty_by_card["Total Enrolled"].sum() * 2*pi
hvs_loyalty_by_card = hvs_loyalty_by_card.assign(Angle=list(angle))
hvs_loyalty_by_card = hvs_loyalty_by_card.assign(Color=color)
hvs_loyalty_by_card["Percent"] = hvs_loyalty_by_card["Total Enrolled"]/total * 100
hvs_loyalty_by_card.rename(columns={"Loyalty Card":"Loyalty_Card"}, inplace=True)

pie_card = figure(title="High Value Segment Breakdown by Loyalty Card Status \n", x_range=(-0.5, 1.0), height=350,
            tools="hover", tooltips="@Loyalty_Card: @Percent%")

pie_card.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum("Angle", include_zero=True), end_angle=cumsum("Angle"),
        line_color="white", fill_color="Color", legend_field="Loyalty_Card", source=hvs_loyalty_by_card)

pie_card.axis.axis_label = None
pie_card.axis.visible = False
pie_card.grid.grid_line_color = None

# Create pie chart for education
color = ["powderblue", "lightskyblue", "dodgerblue", "blue", "navy"]
total = hvs_loyalty_by_ed["Total Enrolled"].sum()
angle = hvs_loyalty_by_ed["Total Enrolled"]/hvs_loyalty_by_ed["Total Enrolled"].sum() * 2*pi
hvs_loyalty_by_ed = hvs_loyalty_by_ed.assign(Angle=list(angle))
hvs_loyalty_by_ed = hvs_loyalty_by_ed.assign(Color=color)
hvs_loyalty_by_ed["Percent"] = hvs_loyalty_by_ed["Total Enrolled"]/total * 100

pie_ed = figure(title="High Value Segment Breakdown by Education \n", x_range=(-0.5, 1.0), height=350,
            tools="hover", tooltips="@Education: @Percent%")

pie_ed.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum("Angle", include_zero=True), end_angle=cumsum("Angle"),
        line_color="white", fill_color="Color", legend_field="Education", source=hvs_loyalty_by_ed)

pie_ed.axis.axis_label = None
pie_ed.axis.visible = False
pie_ed.grid.grid_line_color = None

# Create pie chart for marital status
color = ["lightskyblue", "blue", "navy"]
total = hvs_loyalty_by_ms["Total Enrolled"].sum()
angle = hvs_loyalty_by_ms["Total Enrolled"]/hvs_loyalty_by_ms["Total Enrolled"].sum() * 2*pi
hvs_loyalty_by_ms = hvs_loyalty_by_ms.assign(Angle=list(angle))
hvs_loyalty_by_ms = hvs_loyalty_by_ms.assign(Color=color)
hvs_loyalty_by_ms["Percent"] = hvs_loyalty_by_ms["Total Enrolled"]/total * 100
hvs_loyalty_by_ms.rename(columns={"Marital Status":"Marital_Status"}, inplace=True)

pie_ms = figure(title="High Value Segment Breakdown by Marital Status \n", x_range=(-0.5, 1.0), height=350,
            tools="hover", tooltips="@Marital_Status: @Percent%")

pie_ms.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum("Angle", include_zero=True), end_angle=cumsum("Angle"),
        line_color="white", fill_color="Color", legend_field="Marital_Status", source=hvs_loyalty_by_ms)

pie_ms.axis.axis_label = None
pie_ms.axis.visible = False
pie_ms.grid.grid_line_color = None

show(row(pie_card, pie_ed, pie_ms))

### Analysis of the 2018 Promotion

In [26]:
# Impact of promotion campaign on 2018 enrollment

loyalty_enrollment = pd.DataFrame(loyalty_history.groupby(["Enrollment Year", "Enrollment Type"]).size().reset_index())

# Pivot the loyalty_enrollment to get the required columns
loyalty_enrollment = loyalty_enrollment.pivot(index="Enrollment Year", columns="Enrollment Type", values=0).reset_index().fillna(0)

# Prepare data
year = loyalty_enrollment["Enrollment Year"]
standard = loyalty_enrollment["Standard"]
promotion = loyalty_enrollment["2018 Promotion"]
categories = ["Standard", "2018 Promotion"]
colors = ["green", "blue"]
data = {"years":year, "Standard":standard, "2018 Promotion":promotion}
source = ColumnDataSource(data=data)

# Create stacked bar chart
stacked = figure(title="2012-2018 Enrollment Breakdown by Enrollment Type \n", x_range=year, y_range=(0, 3200),
              x_axis_label='Year', y_axis_label="Number of Loyalty Members", tooltips="$name: @$name",
              width=800, height=500)
stacked.vbar_stack(categories, x='years', width=0.7, color=colors, source=source, legend_label=categories)

# Customize the chart
stacked.legend.location = "top_left"
stacked.legend.orientation = "horizontal"
stacked.legend.label_text_font_size = "9pt"
stacked.xgrid.grid_line_color = None

show(stacked)

In [27]:
# Breakdown of enrollment by loyalty card

# Break enrollment year into two groups, 2012-2017 and 2018
def group_year(year):
    return '2012-2017' if year in ["2012", "2013", "2014", "2015", "2016", "2017"] else '2018'

loyalty_history["Grouped Enrollment Year"] = loyalty_history["Enrollment Year"].apply(group_year)

# Group by Grouped Enrollment Year, Enrollment Type, Loyalty Card
loyalty_by_card = pd.DataFrame(loyalty_history.groupby(["Grouped Enrollment Year", "Enrollment Type", "Loyalty Card"]).size().reset_index())
loyalty_by_card.rename(columns={0:"Total Enrolled"}, inplace=True)

# Prepare data for the pie charts
loyalty_by_card_2012_2017 = loyalty_by_card[loyalty_by_card["Grouped Enrollment Year"] == "2012-2017"]
loyalty_by_card_2018_Standard = loyalty_by_card[(loyalty_by_card["Grouped Enrollment Year"] == "2018") & (loyalty_by_card["Enrollment Type"] == "Standard")]
loyalty_by_card_2018_Promotion = loyalty_by_card[(loyalty_by_card["Grouped Enrollment Year"] == "2018") & (loyalty_by_card["Enrollment Type"] == "2018 Promotion")]

# Create pie chart for 2012 - 2017
color = ["lightskyblue", "blue", "navy"]
total = loyalty_by_card_2012_2017["Total Enrolled"].sum()
angle = loyalty_by_card_2012_2017["Total Enrolled"]/loyalty_by_card_2012_2017["Total Enrolled"].sum() * 2*pi
loyalty_by_card_2012_2017 = loyalty_by_card_2012_2017.assign(Angle=list(angle))
loyalty_by_card_2012_2017 = loyalty_by_card_2012_2017.assign(Color=color)
loyalty_by_card_2012_2017["Percent"] = loyalty_by_card_2012_2017["Total Enrolled"]/total * 100
loyalty_by_card_2012_2017.rename(columns={"Loyalty Card":"Loyalty_Card"}, inplace=True)

pie1 = figure(title="2012-2017 Enrollment Breakdown by Loyalty Card Status \n", x_range=(-0.5, 1.0), height=350,
            tools="hover", tooltips="@Loyalty_Card: @Percent%")

pie1.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum("Angle", include_zero=True), end_angle=cumsum("Angle"),
        line_color="white", fill_color="Color", legend_field="Loyalty_Card", source=loyalty_by_card_2012_2017)

pie1.axis.axis_label = None
pie1.axis.visible = False
pie1.grid.grid_line_color = None

# Create pie chart for 2018 Standard Enrollment
total = loyalty_by_card_2018_Standard["Total Enrolled"].sum()
angle = loyalty_by_card_2018_Standard["Total Enrolled"]/loyalty_by_card_2018_Standard["Total Enrolled"].sum() * 2*pi
loyalty_by_card_2018_Standard = loyalty_by_card_2018_Standard.assign(Angle=list(angle))
loyalty_by_card_2018_Standard = loyalty_by_card_2018_Standard.assign(Color=color)
loyalty_by_card_2018_Standard["Percent"] = loyalty_by_card_2018_Standard["Total Enrolled"]/total * 100
loyalty_by_card_2018_Standard.rename(columns={"Loyalty Card":"Loyalty_Card"}, inplace=True)

pie2 = figure(title="2018 Standard Enrollment Breakdown by Loyalty Card Status \n", x_range=(-0.5, 1.0), height=350,
            tools="hover", tooltips="@Loyalty_Card: @Percent%")

pie2.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum("Angle", include_zero=True), end_angle=cumsum("Angle"),
        line_color="white", fill_color="Color", legend_field="Loyalty_Card", source=loyalty_by_card_2018_Standard)

pie2.axis.axis_label = None
pie2.axis.visible = False
pie2.grid.grid_line_color = None

# Create pie chart for 2018 Promotion Enrollment
total = loyalty_by_card_2018_Promotion["Total Enrolled"].sum()
angle = loyalty_by_card_2018_Promotion["Total Enrolled"]/loyalty_by_card_2018_Promotion["Total Enrolled"].sum() * 2*pi
loyalty_by_card_2018_Promotion = loyalty_by_card_2018_Promotion.assign(Angle=list(angle))
loyalty_by_card_2018_Promotion = loyalty_by_card_2018_Promotion.assign(Color=color)
loyalty_by_card_2018_Promotion["Percent"] = loyalty_by_card_2018_Promotion["Total Enrolled"]/total * 100
loyalty_by_card_2018_Promotion.rename(columns={"Loyalty Card":"Loyalty_Card"}, inplace=True)

pie3 = figure(title="2018 Promotion Enrollment Breakdown by Loyalty Card Status \n", x_range=(-0.5, 1.0), height=350,
            tools="hover", tooltips="@Loyalty_Card: @Percent%")

pie3.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum("Angle", include_zero=True), end_angle=cumsum("Angle"),
        line_color="white", fill_color="Color", legend_field="Loyalty_Card", source=loyalty_by_card_2018_Promotion)

pie3.axis.axis_label = None
pie3.axis.visible = False
pie3.grid.grid_line_color = None

show(row(pie1, pie2, pie3))

In [28]:
# Breakdown of enrollment by gender

# Group by Grouped Enrollment Year, Enrollment Type, Gender
loyalty_by_gender = pd.DataFrame(loyalty_history.groupby(["Grouped Enrollment Year", "Enrollment Type", "Gender"]).size().reset_index())
loyalty_by_gender.rename(columns={0:"Total Enrolled"}, inplace=True)

# Prepare data for the pie charts
loyalty_by_gender_2012_2017 = loyalty_by_gender[loyalty_by_gender["Grouped Enrollment Year"] == "2012-2017"]
loyalty_by_gender_2018_Standard = loyalty_by_gender[(loyalty_by_gender["Grouped Enrollment Year"] == "2018") & (loyalty_by_gender["Enrollment Type"] == "Standard")]
loyalty_by_gender_2018_Promotion = loyalty_by_gender[(loyalty_by_gender["Grouped Enrollment Year"] == "2018") & (loyalty_by_gender["Enrollment Type"] == "2018 Promotion")]

# Create pie chart for 2012 - 2017
color = ["lightskyblue", "blue"]
total = loyalty_by_gender_2012_2017["Total Enrolled"].sum()
angle = loyalty_by_gender_2012_2017["Total Enrolled"]/loyalty_by_gender_2012_2017["Total Enrolled"].sum() * 2*pi
loyalty_by_gender_2012_2017 = loyalty_by_gender_2012_2017.assign(Angle=list(angle))
loyalty_by_gender_2012_2017 = loyalty_by_gender_2012_2017.assign(Color=color)
loyalty_by_gender_2012_2017["Percent"] = loyalty_by_gender_2012_2017["Total Enrolled"]/total * 100

pie1g = figure(title="2012-2017 Enrollment Breakdown by Gender \n", x_range=(-0.5, 1.0), height=350,
            tools="hover", tooltips="@Gender: @Percent%")

pie1g.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum("Angle", include_zero=True), end_angle=cumsum("Angle"),
        line_color="white", fill_color="Color", legend_field="Gender", source=loyalty_by_gender_2012_2017)

pie1g.axis.axis_label = None
pie1g.axis.visible = False
pie1g.grid.grid_line_color = None

# Create pie chart for 2018 Standard Enrollment
total = loyalty_by_gender_2018_Standard["Total Enrolled"].sum()
angle = loyalty_by_gender_2018_Standard["Total Enrolled"]/loyalty_by_gender_2018_Standard["Total Enrolled"].sum() * 2*pi
loyalty_by_gender_2018_Standard = loyalty_by_gender_2018_Standard.assign(Angle=list(angle))
loyalty_by_gender_2018_Standard = loyalty_by_gender_2018_Standard.assign(Color=color)
loyalty_by_gender_2018_Standard["Percent"] = loyalty_by_gender_2018_Standard["Total Enrolled"]/total * 100

pie2g = figure(title="2018 Standard Enrollment Breakdown by Gender \n", x_range=(-0.5, 1.0), height=350,
            tools="hover", tooltips="@Gender: @Percent%")

pie2g.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum("Angle", include_zero=True), end_angle=cumsum("Angle"),
        line_color="white", fill_color="Color", legend_field="Gender", source=loyalty_by_gender_2018_Standard)

pie2g.axis.axis_label = None
pie2g.axis.visible = False
pie2g.grid.grid_line_color = None

# Create pie chart for 2018 Promotion Enrollment
total = loyalty_by_gender_2018_Promotion["Total Enrolled"].sum()
angle = loyalty_by_gender_2018_Promotion["Total Enrolled"]/loyalty_by_gender_2018_Promotion["Total Enrolled"].sum() * 2*pi
loyalty_by_gender_2018_Promotion = loyalty_by_gender_2018_Promotion.assign(Angle=list(angle))
loyalty_by_gender_2018_Promotion = loyalty_by_gender_2018_Promotion.assign(Color=color)
loyalty_by_gender_2018_Promotion["Percent"] = loyalty_by_gender_2018_Promotion["Total Enrolled"]/total * 100

pie3g = figure(title="2018 Promotion Enrollment Breakdown by Gender \n", x_range=(-0.5, 1.0), height=350,
            tools="hover", tooltips="@Gender: @Percent%")

pie3g.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum("Angle", include_zero=True), end_angle=cumsum("Angle"),
        line_color="white", fill_color="Color", legend_field="Gender", source=loyalty_by_gender_2018_Promotion)

pie3g.axis.axis_label = None
pie3g.axis.visible = False
pie3g.grid.grid_line_color = None

show(row(pie1g, pie2g, pie3g))

In [29]:
# Breakdown of enrollment by education

# Group by Grouped Enrollment Year, Enrollment Type, Education
loyalty_by_ed = pd.DataFrame(loyalty_history.groupby(["Grouped Enrollment Year", "Enrollment Type", "Education"]).size().reset_index())
loyalty_by_ed.rename(columns={0:"Total Enrolled"}, inplace=True)
loyalty_by_ed

# Prepare data for the pie charts
loyalty_by_ed_2012_2017 = loyalty_by_ed[loyalty_by_ed["Grouped Enrollment Year"] == "2012-2017"]
loyalty_by_ed_2018_Standard = loyalty_by_ed[(loyalty_by_ed["Grouped Enrollment Year"] == "2018") & (loyalty_by_ed["Enrollment Type"] == "Standard")]
loyalty_by_ed_2018_Promotion = loyalty_by_ed[(loyalty_by_ed["Grouped Enrollment Year"] == "2018") & (loyalty_by_ed["Enrollment Type"] == "2018 Promotion")]

# Create pie chart for 2012 - 2017
color = ["powderblue", "lightskyblue", "dodgerblue", "blue", "navy"]
total = loyalty_by_ed_2012_2017["Total Enrolled"].sum()
angle = loyalty_by_ed_2012_2017["Total Enrolled"]/loyalty_by_ed_2012_2017["Total Enrolled"].sum() * 2*pi
loyalty_by_ed_2012_2017 = loyalty_by_ed_2012_2017.assign(Angle=list(angle))
loyalty_by_ed_2012_2017 = loyalty_by_ed_2012_2017.assign(Color=color)
loyalty_by_ed_2012_2017["Percent"] = loyalty_by_ed_2012_2017["Total Enrolled"]/total * 100

pie1e = figure(title="2012-2017 Enrollment Breakdown by Education \n", x_range=(-0.5, 1.0), height=350,
            tools="hover", tooltips="@Education: @Percent%")

pie1e.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum("Angle", include_zero=True), end_angle=cumsum("Angle"),
        line_color="white", fill_color="Color", legend_field="Education", source=loyalty_by_ed_2012_2017)

pie1e.axis.axis_label = None
pie1e.axis.visible = False
pie1e.grid.grid_line_color = None

# Create pie chart for 2018 Standard Enrollment
total = loyalty_by_ed_2018_Standard["Total Enrolled"].sum()
angle = loyalty_by_ed_2018_Standard["Total Enrolled"]/loyalty_by_ed_2018_Standard["Total Enrolled"].sum() * 2*pi
loyalty_by_ed_2018_Standard = loyalty_by_ed_2018_Standard.assign(Angle=list(angle))
loyalty_by_ed_2018_Standard = loyalty_by_ed_2018_Standard.assign(Color=color)
loyalty_by_ed_2018_Standard["Percent"] = loyalty_by_ed_2018_Standard["Total Enrolled"]/total * 100

pie2e = figure(title="2018 Standard Enrollment Breakdown by Education \n", x_range=(-0.5, 1.0), height=350,
            tools="hover", tooltips="@Education: @Percent%")

pie2e.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum("Angle", include_zero=True), end_angle=cumsum("Angle"),
        line_color="white", fill_color="Color", legend_field="Education", source=loyalty_by_ed_2018_Standard)

pie2e.axis.axis_label = None
pie2e.axis.visible = False
pie2e.grid.grid_line_color = None

# Create pie chart for 2018 Promotion Enrollment
total = loyalty_by_ed_2018_Promotion["Total Enrolled"].sum()
angle = loyalty_by_ed_2018_Promotion["Total Enrolled"]/loyalty_by_ed_2018_Promotion["Total Enrolled"].sum() * 2*pi
loyalty_by_ed_2018_Promotion = loyalty_by_ed_2018_Promotion.assign(Angle=list(angle))
loyalty_by_ed_2018_Promotion = loyalty_by_ed_2018_Promotion.assign(Color=color)
loyalty_by_ed_2018_Promotion["Percent"] = loyalty_by_ed_2018_Promotion["Total Enrolled"]/total * 100

pie3e = figure(title="2018 Promotion Enrollment Breakdown by Education \n", x_range=(-0.5, 1.0), height=350,
            tools="hover", tooltips="@Education: @Percent%")

pie3e.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum("Angle", include_zero=True), end_angle=cumsum("Angle"),
        line_color="white", fill_color="Color", legend_field="Education", source=loyalty_by_ed_2018_Promotion)

pie3e.axis.axis_label = None
pie3e.axis.visible = False
pie3e.grid.grid_line_color = None

show(row(pie1e, pie2e, pie3e))

In [30]:
# Breakdown of enrollment by marital status

# Group by Grouped Enrollment Year, Enrollment Type, Marital Status
loyalty_by_ms = pd.DataFrame(loyalty_history.groupby(["Grouped Enrollment Year", "Enrollment Type", "Marital Status"]).size().reset_index())
loyalty_by_ms.rename(columns={0:"Total Enrolled"}, inplace=True)

# Prepare data for the pie charts
loyalty_by_ms_2012_2017 = loyalty_by_ms[loyalty_by_ms["Grouped Enrollment Year"] == "2012-2017"]
loyalty_by_ms_2018_Standard = loyalty_by_ms[(loyalty_by_ms["Grouped Enrollment Year"] == "2018") & (loyalty_by_ms["Enrollment Type"] == "Standard")]
loyalty_by_ms_2018_Promotion = loyalty_by_ms[(loyalty_by_ms["Grouped Enrollment Year"] == "2018") & (loyalty_by_ms["Enrollment Type"] == "2018 Promotion")]

# Create pie chart for 2012 - 2017
color = ["lightskyblue", "blue", "navy"]
total = loyalty_by_ms_2012_2017["Total Enrolled"].sum()
angle = loyalty_by_ms_2012_2017["Total Enrolled"]/loyalty_by_ms_2012_2017["Total Enrolled"].sum() * 2*pi
loyalty_by_ms_2012_2017 = loyalty_by_ms_2012_2017.assign(Angle=list(angle))
loyalty_by_ms_2012_2017 = loyalty_by_ms_2012_2017.assign(Color=color)
loyalty_by_ms_2012_2017["Percent"] = loyalty_by_ms_2012_2017["Total Enrolled"]/total * 100
loyalty_by_ms_2012_2017.rename(columns={"Marital Status":"Marital_Status"}, inplace=True)

pie1m = figure(title="2012-2017 Enrollment Breakdown by Marital Status \n", x_range=(-0.5, 1.0), height=350,
            tools="hover", tooltips="@Marital_Status: @Percent%")

pie1m.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum("Angle", include_zero=True), end_angle=cumsum("Angle"),
        line_color="white", fill_color="Color", legend_field="Marital_Status", source=loyalty_by_ms_2012_2017)

pie1m.axis.axis_label = None
pie1m.axis.visible = False
pie1m.grid.grid_line_color = None

# Create pie chart for 2018 Standard Enrollment
total = loyalty_by_ms_2018_Standard["Total Enrolled"].sum()
angle = loyalty_by_ms_2018_Standard["Total Enrolled"]/loyalty_by_ms_2018_Standard["Total Enrolled"].sum() * 2*pi
loyalty_by_ms_2018_Standard = loyalty_by_ms_2018_Standard.assign(Angle=list(angle))
loyalty_by_ms_2018_Standard = loyalty_by_ms_2018_Standard.assign(Color=color)
loyalty_by_ms_2018_Standard["Percent"] = loyalty_by_ms_2018_Standard["Total Enrolled"]/total * 100
loyalty_by_ms_2018_Standard.rename(columns={"Marital Status":"Marital_Status"}, inplace=True)

pie2m = figure(title="2018 Standard Enrollment Breakdown by Marital Status \n", x_range=(-0.5, 1.0), height=350,
            tools="hover", tooltips="@Marital_Status: @Percent%")

pie2m.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum("Angle", include_zero=True), end_angle=cumsum("Angle"),
        line_color="white", fill_color="Color", legend_field="Marital_Status", source=loyalty_by_ms_2018_Standard)

pie2m.axis.axis_label = None
pie2m.axis.visible = False
pie2m.grid.grid_line_color = None

# Create pie chart for 2018 Promotion Enrollment
total = loyalty_by_ms_2018_Promotion["Total Enrolled"].sum()
angle = loyalty_by_ms_2018_Promotion["Total Enrolled"]/loyalty_by_ms_2018_Promotion["Total Enrolled"].sum() * 2*pi
loyalty_by_ms_2018_Promotion = loyalty_by_ms_2018_Promotion.assign(Angle=list(angle))
loyalty_by_ms_2018_Promotion = loyalty_by_ms_2018_Promotion.assign(Color=color)
loyalty_by_ms_2018_Promotion["Percent"] = loyalty_by_ms_2018_Promotion["Total Enrolled"]/total * 100
loyalty_by_ms_2018_Promotion.rename(columns={"Marital Status":"Marital_Status"}, inplace=True)

pie3m = figure(title="2018 Promotion Enrollment Breakdown by Marital Status \n", x_range=(-0.5, 1.0), height=350,
            tools="hover", tooltips="@Marital_Status: @Percent%")

pie3m.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum("Angle", include_zero=True), end_angle=cumsum("Angle"),
        line_color="white", fill_color="Color", legend_field="Marital_Status", source=loyalty_by_ms_2018_Promotion)

pie3m.axis.axis_label = None
pie3m.axis.visible = False
pie3m.grid.grid_line_color = None

show(row(pie1m, pie2m, pie3m))

## 5. Insights from the Exploratory Data Analysis

In summary,
*   The 2018 promotion had a positive effect on enrollment of new members into the loyalty program. Enrollment numbers had a jump in 2018 compared to the previous 5 years
*   There is really no difference in how the different demography of the loyalty program members responded to the 2018 promotion
*   Most of the loyalty program members actively book flights with the airline, with activity better in 2018 than 2017
*   Loyalty program members are most active in the summer months (June – August). They are also more active in March and December compared to the remaining months
*   The flight activity of customers classified as high value is similar to that of the general loyalty program members - most active in the summer months (June – August), and more active in March and December compared to the remaining months
*   Also, the demographic breakdown of high value members is similar to that of the general loyalty program members
